<a href="https://colab.research.google.com/github/lucas-larsson/II142X/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# II142X Degree project  VGG16 vs VGG32 a depper dive

- Alex Lindberg : alex5@kth.se
- Lucas Larsson : lulars@kth.se

In [1]:
# make sure we are running NVIDIA GPU V 100, specifically Tesla V100-SXM2-16GB
!nvidia-smi

Sun Apr 21 12:56:17 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Imports and setup

In [2]:
import torch
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Clone the git repos to accsess custom functions

!git clone https://github.com/lucas-larsson/II142X.git
%cd II142X/

In [7]:
# From local helper files

from helpers.evaluation import set_all_seeds, set_deterministic, compute_confusion_matrix
from helpers.training import train_model
from helpers.plotting import plot_training_loss, plot_accuracy, show_examples, plot_confusion_matrix
from helpers.data_importing import get_dataloaders_cifar10, UnNormalize
from helpers.plotting import plot_metric

## Settings and Dataset

In [ ]:
RANDOM_SEED = 123
BATCH_SIZE = 256
NUM_EPOCHS = 50
set_all_seeds(RANDOM_SEED)
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
results_dir = '/content/drive/My Drive/Colab Notebooks/II142X-Results'

In [ ]:
##########################
### CIFAR-10 DATASET
##########################

##  VGG 16, resize to 64x64

train_transforms_64 = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.RandomCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms_64 = torchvision.transforms.Compose([
    torchvision.transforms.Resize((70, 70)),
    torchvision.transforms.CenterCrop((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_loader_64, valid_loader_64, test_loader_64 = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms_64,
    test_transforms=test_transforms_64,
    num_workers=2)

# Checking the dataset
for images, labels in train_loader_64:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break

## VGG 23 resize to 128x128

train_transforms_128 = torchvision.transforms.Compose([
    torchvision.transforms.Resize((140, 140)),
    torchvision.transforms.RandomCrop((128, 128)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transforms_128 = torchvision.transforms.Compose([
    torchvision.transforms.Resize((140, 140)),
    torchvision.transforms.CenterCrop((128, 128)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_loader_128, valid_loader_128, test_loader_128 = get_dataloaders_cifar10(
    batch_size=BATCH_SIZE,
    validation_fraction=0.1,
    train_transforms=train_transforms_128,
    test_transforms=test_transforms_128,
    num_workers=2)

# Checking the dataset
for images, labels in train_loader_128:
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    print('Class labels of 10 examples:', labels[:10])
    break


## Training and Testing

In [ ]:
## VGG 16
from models.VGG_16 import VGG16

model_16 = VGG16(num_classes=10)
model_16 = model_16.to(DEVICE)

optimizer = torch.optim.SGD(model_16.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)
minibatch_loss_list_16, train_acc_list_16, valid_acc_list_16, valid_precision_list_16, valid_recall_list_16, valid_f1_list_16 = train_model(
    model=model_16,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader_64,
    valid_loader=valid_loader_64,
    test_loader=test_loader_64,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

In [ ]:
from models.VGG_23 import VGG23

model_23 = VGG23(num_classes=10)
model_23 = model_23.to(DEVICE)

optimizer = torch.optim.SGD(model_23.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       verbose=True)

minibatch_loss_list_23, train_acc_list_23, valid_acc_list_23, valid_precision_list_23, valid_recall_list_23, valid_f1_list_23 = train_model(
    model=model_23,
    num_epochs=NUM_EPOCHS,
    train_loader=train_loader_128,
    valid_loader=valid_loader_128,
    test_loader=test_loader_128,
    optimizer=optimizer,
    device=DEVICE,
    scheduler=scheduler,
    scheduler_on='valid_acc',
    logging_interval=100)

## Visualizing the results

In [ ]:
## Plotting the results

plot_metric(valid_precision_list_16, valid_precision_list_23, 'Precision', results_dir)
plot_metric(valid_recall_list_16, valid_recall_list_23, 'Recall', results_dir)
plot_metric(valid_f1_list_16 , valid_f1_list_23, 'F1 score', results_dir)
plot_accuracy(train_acc_list_16, valid_acc_list_16, train_acc_list_23, valid_acc_list_23, results_dir)
plot_training_loss(minibatch_loss_list_16, minibatch_loss_list_23, NUM_EPOCHS, len(train_loader_128), results_dir, 100)


In [ ]:
## Confusion matrix and examples VGG 16
model_16.cpu()
unnormalizer = UnNormalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}

show_examples(model=model_16, data_loader=test_loader_64, unnormalizer=unnormalizer, class_dict=class_dict)


mat = compute_confusion_matrix(model=model_16, data_loader=test_loader_64, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())


## this is in testing phase, have not been able to run it yet since i am running on a local machine

y_true, y_pred = compute_confusion_matrix(model_16, test_loader_64, torch.device('cpu'))
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

In [ ]:
## Confusion matrix and examples VGG 23
model_23.cpu()
unnormalizer = UnNormalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
class_dict = {0: 'airplane',
              1: 'automobile',
              2: 'bird',
              3: 'cat',
              4: 'deer',
              5: 'dog',
              6: 'frog',
              7: 'horse',
              8: 'ship',
              9: 'truck'}

show_examples(model=model_23, data_loader=test_loader_128, unnormalizer=unnormalizer, class_dict=class_dict)

mat = compute_confusion_matrix(model=model_23, data_loader=test_loader_128, device=torch.device('cpu'))
plot_confusion_matrix(mat, class_names=class_dict.values())

## this is in testing phase, have not been able to run it yet since i am running on a local machine

y_true, y_pred = compute_confusion_matrix(model_23, test_loader_128, torch.device('cpu'))
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

## Saving the results

In [ ]:
results = pd.DataFrame({'VGG16': [valid_precision_list_16[-1], valid_recall_list_16[-1], valid_f1_list_16[-1]],
                        'VGG23': [valid_precision_list_23[-1], valid_recall_list_23[-1], valid_f1_list_23[-1]]},
                       index=['Precision', 'Recall', 'F1'])
results.to_csv(results_dir + '/final_results.csv')